**Mock Abstract Generator**

Bu model **Cem Uzunoglu**'nun (https://github.com/cemkod) oluşturduğu bir repo'dan türetilmiştir. Türkçe ön-eğitim için **Görkem Göknar**'ın modeli, ingilizce ön-eğitim için **OpenAi**'ın gpt2 base modelinden yararlanılmıştır.

**Ş. Barış Terzi**

Modeli eğitmek için runtime/change runtime type'tan GPU'nun seçilmelidir.

Kullanılanacak kütüphaneleri;
- **Transformers**, hazır modeller içeren ve modelleri fine-tune etmeyi kolaylaştıran bir kütüphane
- **PyTorch Transformer**'ın arkaplanda modeli eğitmek için kullandığı machine learning kütüphanesi
- **ipywidget**s'ı da ekrana textbox, buton gibi şeyleri çizdirmekte kullanıyoruz


In [1]:
!pip install --upgrade pip
!pip install transformers
!pip install torch
!pip install ipywidgets

     |████████████████████████████████| 1.5MB 7.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 2.1 MB 8.1 MB/s 
     |████████████████████████████████| 883 kB 58.0 MB/s 
     |████████████████████████████████| 3.2 MB 56.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893258 sha256=78d4b957435c027f0851041a50802770ab0ab308ccbb74259708b212c357dd2f
  Stored in directory: /root/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses


Eğittiğimiz modeli google drive'da saklanabilmesi için google drive'a erişim izni verilmesi gerekmektedir.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


İstenilen dildeki hücre aktif hale, diğeri pasif hale getirilmelidir. Bu işlem ön-eğitimdir(pre-train) ve temel dil bilgisi verilerini modelin öğrenmesini sağlar.

In [ ]:
#Türkçe eğitim için 

#from transformers import AutoTokenizer, AutoModelWithLMHead
#import torch

#tokenizer = AutoTokenizer.from_pretrained("gorkemgoknar/gpt2-small-turkish")
#model = AutoModelWithLMHead.from_pretrained("gorkemgoknar/gpt2-small-turkish")
# GPT-2 en fazla 1024 tokenlik bir dizi uretebiliyor
#tokenizer.model_max_length=1024 


In [ ]:
#İngilizce eğitim için 

from transformers import GPT2Tokenizer, AutoModelWithLMHead #TFGPT2Model
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelWithLMHead.from_pretrained('gpt2')
# GPT-2 en fazla 1024 tokenlik bir dizi uretebiliyor
tokenizer.model_max_length=1024 

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Bu hücreyi çalıştırmadan önce drive'ınıza eğitmek istenilen verilerden oluşan dosya yüklenmeli ve yolu aşağıdaki **"train_path"** ve **"test_path"** değişkenlerine tanımlanmalıdır.

Eğitim verisi txt dosyası olarak yüklenmelidir. Eğitim için kullanılacak özetler bu dosyaya aralarında "**~**" işareti olacak biçimde yerleştirilmelidir. Bu işaret bir özetin bitip diğerinin başladığını tanımlar. 

Veri miktarının artması modelin daha verimli olmasını sağlayacaktır. Eğitim datası en az 200 KB, ideal olarak 500 KB olmalıdır.

In [ ]:
#from transformers import AutoTokenizer  #Türkçe eğitim için
from transformers import GPT2Tokenizer  #İngilizce eğitim için 

train_path = 'drive/MyDrive/abstractGen/abstract1.txt'
test_path = 'drive/MyDrive/abstractGen/abstract1.txt'

from transformers import TextDataset,DataCollatorForLanguageModeling
 
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
           tokenizer=tokenizer,
           file_path=test_path,
           block_size=128)
 
    data_collator = DataCollatorForLanguageModeling(
     tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator
 
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


Bu satırda eğitim paraametreleri tanımlanır.  
**num_train_epochs** parametresi epoch sayısını tanımlar. Epoch ile eğitim verisinin model tarafından kaç defa değerlendirileceği belirlenir. 

Eğitilen bilgisayarın ekran kartı kapasitesine göre **per_device_train_batch_size** ve **per_device_eval_batch_size** değerleri değiştirilebilir.


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

training_args = TrainingArguments(
    output_dir="./gpt2-abs", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    #learning_rate=0.00001,
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 100, # Number of update steps between two evaluations.
    logging_steps = 10,
    save_steps=1000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Bu hücre ile eğitim süreci başlatılır. **loss** değeri artık düşmediğinde daha fazla eğitilemediği anlamına gelir.

In [ ]:
trainer.train()

Step,Training Loss
10,4.464000
20,4.433500
30,4.400300
40,4.335300
50,4.258200
60,4.236200
70,4.164800
80,4.108200
90,4.072000
100,3.981100


TrainOutput(global_step=1400, training_loss=1.643473390851702, metrics={'train_runtime': 1547.5151, 'train_samples_per_second': 0.905, 'total_flos': 4166842082918400, 'epoch': 100.0})

Bu hücre eğitilen modelin drive'a kaydedilmesini sağlar. İstenilen klasör tanımlanabilir. Kaydedilen model ile algoritma yeniden eğitilmeye gerek kalmadan kullanılabilir.

In [ ]:
model.save_pretrained("drive/MyDrive/abstractGen/gpt2-abs-3epoch")

Bu hücre ile oluşturulan model geri yüklenilir. Tekrar eğitim yapılmak istenmemesi durumunda, google drive yetkilendirme hücresi çalıştırıldıktan sonra bu hücre çalıştırılabilir.

In [5]:
#from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2Tokenizer, AutoModelWithLMHead
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelWithLMHead.from_pretrained("drive/MyDrive/abstractGen/gpt2-abs-3epoch")
# GPT-2 en fazla 1024 tokenlik bir dizi uretebiliyor
tokenizer.model_max_length=1024 


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1010: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Bu hücre ile "abstract" üretimine başlanılabilir. Açılan textBox'a "~" işaretinden sonra üretilecek metnin başlangıcı yazılıp "Generate!" butonuna tıklanmalıdır. Metin uzunluğu belirlemek için **"max_length"** ve **"min_length"** parametreleri değiştirilebilir.

In [6]:
from ipywidgets import widgets
model.eval();
# input sequence

textui = widgets.Textarea(
    value='~\n',
    placeholder='Type something',
    description='String:',
    disabled=False
)

button = widgets.Button(
    description='Generate!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
display(textui)
display(button)

def handle_submit(sender):
  print("Generating!\n")
  text = textui.value
  inputs = tokenizer(text, return_tensors="pt")

  # model output using Top-k sampling text generation method
  sample_outputs = model.generate(inputs.input_ids,
                                  pad_token_id=50256,
                                  do_sample=True, 
                                  max_length=300, # put the token number you want
                                  min_length=200,
                                  top_k=40, 
                                  #top_p=0.95, 
                                  temperature=0.95,
                                  num_return_sequences=1)

  # generated sequence
  for i, sample_output in enumerate(sample_outputs):
      print(">> Generated text {}\n\n{}\nend text".format(i+1, tokenizer.decode(sample_output.tolist())))



button.on_click(handle_submit)


# >> Generated text
#    


Textarea(value='~\n', description='String:', placeholder='Type something')

Button(description='Generate!', icon='check', style=ButtonStyle(), tooltip='Click me')

Generating!

>> Generated text 1

~
The function of paper tapes in monumental structures
referred to in the field of architecture, this research explores through professional case studies how digitisation of architectural paper tapes can be seamlessly incorporated into the production of architectural digitisation. Based on a re-defined history of analogue recording technology relevant to the field of architectural heritage, this paper documents the development of the bit-to-bit conversion mechanism and its corresponding methodology for analogue recording of architectural heritage. A normative research method defined as a capture-and-export system, the analogue recording of architectural heritage is aimed to support the formalization of dealing with the complexities of digitalisation and the production of architectural artifacts. The paper discusses the potential of the application of this technology to support the formalization of architectural heritage and highlights the limitations o